In [1]:
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from pathlib import Path
import requests
import os
import time

In [3]:
target_url = 'https://mp.weixin.qq.com/s/UYTSgfghKDHJE4iBozOBQQ'
sub_targets = ['蜡笔小新大电影合集', '蜡笔小新第一季', '蜡笔小新第二季', '蜡笔小新第三季', '蜡笔小新第四季', '蜡笔小新第五季', '蜡笔小新第六季', '蜡笔小新第七季', '蜡笔小新第八季', '蜡笔小新第九季', '蜡笔小新第十季']
chrome_options = webdriver.chrome.options.Options()
chrome_options.add_argument("--headless")
browser = webdriver.Chrome(options=chrome_options)
browser.get(target_url)
html_source = browser.page_source
soup = BeautifulSoup(html_source, 'html.parser')
Path('urls').mkdir(parents=True, exist_ok=True)
with open('urls/links.txt', 'w') as f:
    for sub_target in sub_targets:
        tag = soup.find('a', textvalue=re.compile('\s*'.join(list(sub_target))))
        f.write(sub_target + ' ' + tag['href'] + '\n')
browser.quit()

In [2]:
# Scroll to the bottom of the page
def scroll_to_bottom(browser):
    last_height = browser.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll to the bottom of the page
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for new content to load (you might need to adjust the sleep time)
        time.sleep(1)

        # Calculate new page height and compare with the previous height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break  # No more new content loaded, exit the loop
        last_height = new_height

In [14]:
with open('urls/links.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        name, url = line.split()
        # browser = webdriver.Chrome(options=chrome_options)
        browser = webdriver.Chrome() # For debugging
        browser.get(url)
        scroll_to_bottom(browser)
        soup = BeautifulSoup(browser.page_source, 'html.parser')
        tags = soup.find_all('li', attrs={"data-title": re.compile('蜡笔小新.*')})
        if_a_tags = False
        if len(tags) == 0:
            tags = soup.find_all('a', text=re.compile('蜡笔小新.*'))
            if_a_tags = True
        with open(f'urls/{name}.txt', 'w') as f_sub:
            for tag in tags:
                if if_a_tags:
                    f_sub.write(re.sub('\s', '', tag.text) + ' ' + tag['href'] + '\n')
                else:
                    f_sub.write(re.sub('\s', '', tag['data-title']) + ' ' + tag['data-link'] + '\n')
        browser.quit()

C:\Users\Jack\AppData\Local\Temp\ipykernel_14784\257698692.py:13: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tags = soup.find_all('a', text=re.compile('蜡笔小新.*'))


In [5]:
from tqdm import tqdm

headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.3 Safari/605.1.15"
}

def download_video_from_url(url, destination_filename):
    response = requests.get(url, stream=True, headers=headers)
    response.raise_for_status()  # Raise an exception for HTTP errors
    total_size_in_bytes= int(response.headers.get('content-length', 0))
    chunk_size=4*1024*1024 # 4MB chunks

    with open(destination_filename, 'wb') as file, tqdm(
        desc=destination_filename,
        total=total_size_in_bytes,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size):
            file.write(chunk)
            bar.update(len(chunk))
            
    print(f"Video downloaded and saved as {destination_filename}")

In [6]:
import glob


paths = glob.glob('urls/蜡笔小新*.txt') # Skip movie
Path('videos').mkdir(parents=True, exist_ok=True)

for path in paths:
    with open(path, 'r') as file:
        # get current file name
        title = os.path.basename(path).split('.')[0]
        Path('videos/' + title).mkdir(parents=True, exist_ok=True)
        lines = file.readlines()
        for line in lines:
            name, url = line.split()
            if os.path.exists('videos/' + title + '/' + name + '.mp4'):
                # print(f"File {'videos/' + title + '/' + name + '.mp4'} already exists. Skipping download.")
                continue
            chrome_options = webdriver.chrome.options.Options()
            chrome_options.add_argument("--headless")
            browser = webdriver.Chrome(options=chrome_options)
            browser.get(url)

            # Wait for JavaScript to render (can use explicit waits here if needed)
            html_source = browser.page_source
            soup = BeautifulSoup(html_source, 'html.parser')
            browser.quit()

            # Find the video tag with the desired src attribute pattern
            video_tag = soup.find('video', src=re.compile(r'^http://mpvideo.qpic.cn/.*'))

            if video_tag:
                video_src = video_tag['src']
                # download video
                download_video_from_url(video_src, 'videos/' + title + '/' + name + '.mp4')
            else:
                print("No matching video tag found.")
        

videos/蜡笔小新第九季/蜡笔小新第九季114把涂鸦擦掉哦.mp4: 100%|██████████| 81.7M/81.7M [01:08<00:00, 1.26MB/s]


Video downloaded and saved as videos/蜡笔小新第九季/蜡笔小新第九季114把涂鸦擦掉哦.mp4


videos/蜡笔小新第九季/蜡笔小新第九季115全身湿答答的园长先生.mp4: 100%|██████████| 81.8M/81.8M [01:45<00:00, 815kB/s]


Video downloaded and saved as videos/蜡笔小新第九季/蜡笔小新第九季115全身湿答答的园长先生.mp4


videos/蜡笔小新第九季/蜡笔小新第九季116艰难的购物.mp4: 100%|██████████| 81.5M/81.5M [01:36<00:00, 889kB/s]


...

videos/蜡笔小新第九季/蜡笔小新第九季台配130风间的坚持哦.mp4: 100%|██████████| 81.6M/81.6M [01:32<00:00, 929kB/s]


Video downloaded and saved as videos/蜡笔小新第九季/蜡笔小新第九季台配130风间的坚持哦.mp4


videos/蜡笔小新第九季/蜡笔小新第九季台配131在夏天的晚上进行天文观测哦.mp4: 100%|██████████| 81.4M/81.4M [01:44<00:00, 820kB/s]


Video downloaded and saved as videos/蜡笔小新第九季/蜡笔小新第九季台配131在夏天的晚上进行天文观测哦.mp4


videos/蜡笔小新第九季/蜡笔小新第九季台配132我是目击者哦.mp4:   0%|          | 0.00/81.7M [00:00<?, ?B/s]